In [101]:
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

# Load and initialize the BirdNET-Analyzer models.
analyzer = Analyzer()
bfile = "bird-voices.mp3"
recording = Recording(
    analyzer,
    bfile,
    lat=51.41778,
    lon=5.462251,
    date=datetime(year=2022, month=5, day=10), # use date or week_48
    min_conf=0.25,
)
recording.analyze()
print(recording.detections)

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.
read_audio_data
read_audio_data: complete, read  34 chunks.
analyze_recording bird-voices.mp3
recording has lon/lat
set_predicted_species_list_from_position
return_predicted_species_list
18
151 species loaded.
[{'common_name': 'Willow Warbler', 'scientific_name': 'Phylloscopus trochilus', 'start_time': 6.0, 'end_time': 9.0, 'confidence': 0.4909761846065521, 'label': 'Phylloscopus trochilus_Willow Warbler'}, {'common_name': 'Willow Warbler', 'scientific_name': 'Phylloscopus trochilus', 'start_time': 21.0, 'end_time': 24.0, 'confidence': 0.42529934644699097, 'label': 'Phylloscopus trochilus_Willow Warbler'}, {'common_name': 'European Robin', 'scientific_name': 'Erithacus rubecula', 'start_time': 24.0, 'end_time': 27.0, 'confidence': 0.6579822301864624, 'label': 'Erithacus rubecula_European Robin'}, {'common_name': 'European Robin', 'scientific_name': 'Erithacus rubecula', 'start_time': 

In [100]:

import os
import pandas as pd

def find_files(root_folder):
    """
    Finds the metadata file and audio files in the specified root folder.
    The root folder is assumed to contain subfolders with audio files and metadata file.
    The metadata file is assumed to be named '...metadata.xlsx' and the audio files are assumed to have the extension '.wav'.
    parameters:
    - root_folder: The path to the root folder containing the audio files and metadata file.
    returns:
    - metadata_filepath: The path to the metadata file.
    - audio_files: A list of relative paths to the audio files.

    """
    audio_files = []
    metadata_filepath = None
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith('metadata.xlsx'):
                metadata_filepath = os.path.join(dirpath, filename)
                print(f"Found metadata file: {metadata_filepath}")

            elif filename.endswith('.wav'):
                audio_files.append(os.path.join(dirpath, filename))
            

    return metadata_filepath, audio_files

In [99]:


import pandas as pd
import os

def add_labels(metadata_file_path, results,column_name):
    """
    Adds the BirdNET labels to the metadata file.
    parameters:
    - metadata_file_path: The relative path to the metadata file.
    - results: A list of tuples containing the filepath and BirdNET label.

    """
    # Create a DataFrame from results
    labels_df = pd.DataFrame(results, columns=['filepath', column_name])
    
    # Extract basenames for comparison
    labels_df['filename'] = labels_df['filepath'].apply(os.path.basename)
    
    # Load the existing metadata DataFrame
    df = pd.read_excel(metadata_file_path)
    
    # Ensure filenames are basenames for comparison
    df['filename'] = df['filename'].apply(os.path.basename)
    
    # Check if the 'BirdNET_label' column exists; if not, create it with NaN values
    if column_name not in df.columns:
        df[column_name] = pd.NA
    
    # Merge the existing df with the labels_df
    df = df.merge(labels_df[['filename', column_name]], on='filename', how='left', suffixes=('', '_new'))
    
    # Update the 'BirdNET_label' column with new values where available
    df[column_name] = df[column_name+'_new'].combine_first(df[column_name])
    
    # Drop the temporary 'BirdNET_label_new' column
    df.drop('BirdNET_label_new', axis=1, inplace=True)
 
    # Construct the new filename
    original_filename = os.path.basename(metadata_file_path)
    new_filename = original_filename.replace('metadata', column_name+'_metadata')
    new_filepath = os.path.join(os.path.dirname(metadata_file_path), new_filename)
    # Save the updated DataFrame to a new Excel file
    df.to_excel(new_filepath, index=False)
    
    


In [98]:
def analyze_files_birdnet(list_files_paths):  
    """Analyzer for BirdNET
    parameters:
    - list_files_paths: List of file paths to analyze
    returns:
    - results: A list of tuples containing the filepath and BirdNET label.
    """  
    results = []
    for file in list_files_paths:
        try:
            recording = Recording(
                analyzer,
                file,
                lat=51.41778,
                lon=5.462251,
                date=datetime(year=2022, month=5, day=10), # use date or week_48
                min_conf=0.25,
            )
            recording.analyze()
            if len(recording.detections) > 0:
                results.append((file, recording.detections[0]['label']))
        except Exception as e:
            print(f"Error processing file {file}: {e}")
    return results

In [97]:
# new pipeline for birdnet processing
def pipeline_birdnet(folder_path):
    """
    Pipeline for processing audio files with BirdNET.
    parameters:
    - folder_path: The path to the root folder containing the audio files and metadata file.
    """
    metadata_file_path, audio_files_paths = find_files(folder_path)
    results = analyze_files_birdnet(audio_files_paths)
    add_labels(metadata_file_path, results, 'BirdNET')   

    

In [95]:
gatden_folder = '../garden_19102023'
pipeline_birdnet(gatden_folder)


Found metadata file: ../garden_19102023/garden_19102023_metadata.xlsx
Found metadata file: ../garden_19102023/garden_19102023_birdnet_metadata.xlsx
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording er_file_2023_10_19_16_55_54.wav
recording has lon/lat
set_predicted_species_list_from_position
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording er_file_2023_10_19_17_18_42.wav
recording has lon/lat
set_predicted_species_list_from_position
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording er_file_2023_10_19_16_41_08.wav
recording has lon/lat
set_predicted_species_list_from_position
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording er_file_2023_10_19_16_39_04.wav
recording has lon/lat
set_predicted_species_list_from_position
read_audio_data
read_audio_data: complete, read  1 chunks.
analyze_recording er_file_2023_10_19_16_59_41.wav
recording has lon/lat
set_predicted_species_list_from_p